In [20]:
import torch
import torch.nn.functional as F
from torch import nn
from torchvision.transforms import CenterCrop
from dataclasses import dataclass

In [21]:
@dataclass
class DiffusionConfig:
    time_embedding_dim: int = 320

In [22]:
class SelfAttention(nn.Module):
    """
    Self Attention mechanism for sequence data.
    
    Attributes:
        scale (float): Scaling factor for the attention scores.
        n_heads (int): Number of attention heads.
        d_head (int): Dimension of each attention head.
        QKV (nn.Linear): Linear layer for Query, Key, Value.
        O (nn.Linear): Linear output layer.
    """
    
    def __init__(self, d_embed, n_heads: int = 4, qkv_bias=True, out_bias=True) -> None:
        """
        Initializes the SelfAttention class.
        
        Args:
            d_embed (int): Dimension of the embedding.
            n_heads (int): Number of attention heads. Defaults to 4.
            qkv_bias (bool): If True, adds bias to QKV linear layer. Defaults to True.
            out_bias (bool): If True, adds bias to O linear layer. Defaults to True.
        """
        
        super().__init__()
        
        self.n_heads = n_heads
        self.d_head = d_embed // n_heads
        self.scale = self.d_head ** -0.5
        self.QKV = nn.Linear(d_embed, d_embed * 3, bias=qkv_bias)
        self.O = nn.Linear(d_embed, d_embed, bias=out_bias)
        
    def forward(self, x: torch.Tensor, mask: bool = False) -> torch.Tensor:
        """
        Forward pass for the SelfAttention mechanism.
        
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, seq_len, d_embed).
            mask (bool): If True, applies the attention mask. Defaults to False.

        Returns:
            torch.Tensor: Processed tensor.
        """
        
        # x: (batch_size, height*width, channels)
        # x: (batch_size, seq_len, d_embed)
        in_shape = x.shape
        bs, seq_len, d_embed = x.shape
        q, k, v = self.QKV(x).chunk(3, dim=-1) # (batch_size, seq_len, d_embed)@(d_embed, d_embed*3) -> (batch_size, seq_len, d_embed*3) -> (3x) (batch_size, seq_len, d_embed)
        
        # (batch_size, seq_len, d_embed) -> (batch_size, seq_len, n_heads, d_head) -> (batch_size, n_heads, seq_len, d_head)
        q = q.view(bs, seq_len, self.n_heads, self.d_head).transpose(1, 2)
        k = k.view(bs, seq_len, self.n_heads, self.d_head).transpose(1, 2)
        v = v.view(bs, seq_len, self.n_heads, self.d_head).transpose(1, 2)
        
        attn_scores = (q @ k.transpose(-2, -1)) * self.scale
        if not mask:
            mask = torch.ones_like(attn_scores).bool().triu(1) # (batch_size, n_heads, seq_len, seq_len)
            attn_scores.masked_fill_(mask, -1e9)
        weights = F.softmax(attn_scores, dim=-1)
        output = weights @ v # (batch_size, n_heads, seq_len, seq_len) -> (batch_size, n_heads, seq_len, d_head) -> (batch_size, n_heads, seq_len, d_head)
        output = output.transpose(1, 2).contiguous().view(in_shape) # (batch_size, n_heads, seq_len, d_head) -> (batch_size, seq_len, n_heads, d_head) -> (batch_size, seq_len, d_embed)
        return self.O(output) # (batch_size, seq_len, d_embed)@(d_embed, d_embed) -> (batch_size, seq_len, d_embed)
    
    
    
class CrossAttention(nn.Module):
    """
    Cross Attention mechanism for sequence data.
    
    Attributes:
        scale (float): Scaling factor for the attention scores.
        n_heads (int): Number of attention heads.
        d_head (int): Dimension of each attention head.
        QKV (nn.Linear): Linear layer for Query, Key, Value.
        O (nn.Linear): Linear output layer.
    """
    
    def __init__(self, n_heads: int, d_embed: int, d_cross: int, qkv_bias=True, out_bias=True) -> None:
        """
        Initializes the SelfAttention class.
        
        Args:
            d_embed (int): Dimension of the embedding.
            n_heads (int): Number of attention heads. Defaults to 4.
            qkv_bias (bool): If True, adds bias to QKV linear layer. Defaults to True.
            out_bias (bool): If True, adds bias to O linear layer. Defaults to True.
        """
        
        super().__init__()
        
        self.n_heads = n_heads
        self.d_head = d_embed // n_heads
        self.scale = self.d_head ** -0.5
        self.Q = nn.Linear(d_embed, d_embed, bias=qkv_bias)
        self.K = nn.Linear(d_cross, d_embed, bias=qkv_bias)
        self.V = nn.Linear(d_cross, d_embed, bias=qkv_bias)
        self.O = nn.Linear(d_embed, d_embed, bias=out_bias)
        
    def forward(self, x_q: torch.Tensor, x_kv: torch.TensorType) -> torch.Tensor:
        """
        Forward pass for the CrossAttention mechanism.
        
        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, seq_len, d_embed).
            mask (bool): If True, applies the attention mask. Defaults to False.

        Returns:
            torch.Tensor: Processed tensor.
        """
        
        # x_q(latents): (batch_size, seq_len_Q, d_embed_Q)
        # x_kv(context): (batch_size, seq_len_KV, d_embed_KV)
        in_shape = x_q.shape
        bs, seq_len, d_embed = x_q.shape
        
        # (batch_size, seq_len, d_embed) -> (batch_size, seq_len, n_heads, d_head) -> (batch_size, n_heads, seq_len, d_head)
        q = self.Q(x_q).view(bs, -1, self.n_heads, self.d_head).transpose(1, 2)
        k = self.K(x_kv).view(bs, -1, self.n_heads, self.d_head).transpose(1, 2)
        v = self.V(x_kv).view(bs, -1, self.n_heads, self.d_head).transpose(1, 2)
        
        attn_scores = (q @ k.transpose(-2, -1)) * self.scale
        weights = F.softmax(attn_scores, dim=-1)
        output = weights @ v # (batch_size, n_heads, seq_len, seq_len) -> (batch_size, n_heads, seq_len, d_head) -> (batch_size, n_heads, seq_len, d_head)
        output = output.transpose(1, 2).contiguous().view(in_shape) # (batch_size, n_heads, seq_len, d_head) -> (batch_size, seq_len, n_heads, d_head) -> (batch_size, seq_len, d_embed)
        return self.O(output) # (batch_size, seq_len, d_embed)@(d_embed, d_embed) -> (batch_size, seq_len, d_embed)

In [23]:
class Unet(nn.Module):
    """
    A PyTorch implementation of the U-Net architecture.
    """
    
    def __init__(self) -> None:
        super().__init__()
        
        self.downs = nn.ModuleList([])
        self.ups = nn.ModuleList([])

In [24]:
unet = Unet()
unet

Unet(
  (downs): ModuleList()
  (ups): ModuleList()
)

In [25]:
class Block(nn.Module):
    
    def __init__(self, in_channels: int, out_channels: int) -> None:
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3) # in_chxout_ch
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3) # out_chxout_ch
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        return x
    
# enc_block1 = Block(in_channels=1, out_channels=64)
# x =  torch.randn(1, 1, 64, 64)
# print(enc_block1(x).shape)

class Encoder(nn.Module):
    
    def __init__(self, channels: tuple = (3, 64, 128, 256, 512, 1024)) -> None:
        super().__init__()
        
        self.enc_blocks = nn.ModuleList(Block(in_ch, out_ch) for in_ch, out_ch in zip(channels, channels[1:]))
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        features = []
        for block in self.enc_blocks:
            x = block(x)
            features.append(x)
            x = self.pool(x)
        return features
        
        
encoder = Encoder().to('mps')
x = torch.randn(1, 3, 572, 572).to('mps')
ftrs = encoder(x)
# for ftr in ftrs: print(ftr.shape)

# an upsampling of the feature map
# a 2x2 convolution (“up-convolution”) that halves the number of feature channels, 
# a concatenation with the correspondingly cropped feature map from the contracting path
# two 3x3 convolutions, each followed by a ReLU

class Decoder(nn.Module):
    def __init__(self, channels=(1024, 512, 256, 128, 64)):
        super().__init__()
        self.channels = channels
        self.up_convs = nn.ModuleList(nn.ConvTranspose2d(in_ch, out_ch, kernel_size=2, stride=2) 
                                      for in_ch, out_ch in zip(channels, channels[1:]))
        self.dec_blocks = nn.ModuleList(Block(in_ch, out_ch) for in_ch, out_ch in zip(channels, channels[1:]))
        
    def forward(self, x: torch.Tensor, features: list) -> torch.Tensor:
        for up_conv, dec_block, feature in zip(self.up_convs, self.dec_blocks, features[::-1][1:]):
            x = up_conv(x)
            enc_feature = CenterCrop(x.shape[2:])(feature)
            x = torch.cat([x, enc_feature], dim=1)
            x = dec_block(x)
        return x
    
decoder = Decoder().to('mps')
x = torch.randn(1, 1024, 28, 28).to('mps')
print(decoder(x, ftrs).shape)

# >> (torch.Size([1, 64, 388, 388])

torch.Size([1, 64, 388, 388])


In [26]:
class UNet(nn.Module):
    def __init__(self, enc_chs=(3,64,128,256,512,1024), dec_chs=(1024, 512, 256, 128, 64), num_class=1, retain_dim=False, out_sz=(572,572)):
        super().__init__()
        self.encoder     = Encoder(enc_chs)
        self.decoder     = Decoder(dec_chs)
        self.head        = nn.Conv2d(dec_chs[-1], num_class, 1)
        self.retain_dim  = retain_dim
        self.out_sz      = out_sz

    def forward(self, x):
        enc_ftrs = self.encoder(x)
        out      = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
        out      = self.head(out)
        if self.retain_dim:
            out = F.interpolate(out, self.out_sz)
        return out

In [27]:
class TimeEmbeddings(nn.Module):
    """
    A PyTorch implementation of the time embeddings used in Diffusion model.
    """
    def __init__(self, time_embedding_dim: int) -> None:
        super().__init__()
        
        self.linear_1 = nn.Linear(in_features=time_embedding_dim, out_features=time_embedding_dim*4) # 320x1280
        self.linear_2 = nn.Linear(in_features=time_embedding_dim*4, out_features=time_embedding_dim*4) # 1280x1280
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.linear_2(F.silu(self.linear_1(x))) # (1, 320) -> (1, 1280)


In [28]:
class Head(nn.Module):
    """
    A PyTorch implementation of the U-Net head.
    """
    def __init__(self, in_channels: int, out_channels: int) -> None:
        super().__init__()
        
        self.group_norm = nn.GroupNorm(num_groups=32, num_channels=in_channels)
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: (batch_size, in_channels, height, width) Bx320x64x64
        x = F.silu(self.group_norm(x))
        return self.conv(x) # Bx320x64x64 -> Bx4x64x64

In [29]:
class Upsample(nn.Module):
    
    def __init__(self, in_channels: int) -> None:
        super().__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=in_channels, kernel_size=3, padding=1)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x:  BxCxHxW -> BxCx2*Hx2*W
        x = F.interpolate(x, scale_factor=2, mode='nearest')
        return self.conv(x)


In [30]:
class UnetAttentionBlock(nn.Module):
    def __init__(self, n_heads: int, n_embed: int, hidden_size: int = 768) -> None:
        super().__init__()
        
        channel_embed = n_heads * n_embed
        self.group_norm = nn.GroupNorm(num_groups=32, num_channels=channel_embed)
        self.conv_in = nn.Conv2d(in_channels=channel_embed, out_channels=channel_embed, kernel_size=1)
        self.attention_1 = SelfAttention(n_heads=n_heads, d_embed=channel_embed, qkv_bias=False)
        self.linear_1 = nn.Linear(in_features=channel_embed, out_features=channel_embed*4*2)
        self.linear_2 = nn.Linear(in_features=channel_embed*4, out_features=channel_embed)
        self.attention_2 = CrossAttention(n_heads=n_heads, d_embed=channel_embed, d_cross=hidden_size, qkv_bias=False) 
        self.layer_norm_1 = nn.LayerNorm(channel_embed)
        self.layer_norm_2 = nn.LayerNorm(channel_embed)
        self.layer_norm_3 = nn.LayerNorm(channel_embed)
        self.conv_out = nn.Conv2d(in_channels=channel_embed, out_channels=channel_embed, kernel_size=1)
        
    def ffn(self, x: torch.Tensor) -> torch.Tensor:
        residue = x
        x, gate = self.linear_1(x).chunk(2, dim=-1)
        x = x * F.gelu(gate)
        return residue + self.linear_2(x)
        

    def forward(self, x: torch.Tensor, context: torch.Tensor) -> torch.Tensor:
        # x: (batch_size, n_heads*n_embed, height, width)
        # context: (batch_size, seq_len, hidden_size)
        n, c, h, w = x.shape
        long_residue = x
        x = self.group_norm(x)
        x = self.conv_in(x)
        
        x = x.view((n, c, h*w)).transpose(-1, -2) # (batch_size, c, height*width) -> (batch_size, height*width, c)
        
        # Layer Normalization + Self-Attention + Residual
        x = x + self.attention_1(self.layer_norm_1(x)) # (batch_size, height*width, c)
        
        # Layer Normalization + Cross-Attention + Residual
        x = x + self.attention_2(self.layer_norm_2(x), context) # (batch_size, height*width, c)
        
        # Layer Normalization + Feed Forward + Residual
        x = self.ffn(self.layer_norm_3(x)) # (batch_size, height*width, c)
        
        x = x.transpose(-1, -2).view((n, c, h, w)) # (batch_size, height*width, c) -> (batch_size, c, height, width)
        
        return self.conv_out(x) + long_residue # (batch_size, c, height, width)

In [31]:
class UnetResidualBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, time_embed_dim: int = 1280) -> None:
        super().__init__()
        
        self.group_norm = nn.GroupNorm(num_groups=32, num_channels=in_channels)
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1)
        self.linear = nn.Linear(in_features=time_embed_dim, out_features=out_channels)
        
        self.group_norm_merged = nn.GroupNorm(num_groups=32, num_channels=out_channels)
        self.conv_merged = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, padding=1)
        
        if in_channels == out_channels:
            self.skip = nn.Identity()
        else:
            self.skip = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1)
            
    def forward(self, x: torch.Tensor, time: torch.Tensor) -> torch.Tensor:
        # x: (batch_size, in_channels, height, width) -> (batch_size, out_channels, height, width)
        residue = x
        x = self.conv(F.silu(self.group_norm(x)))
        t = self.linear(F.silu(time)).unsqueeze(-1).unsqueeze(-1)
        merged = x + t
        return self.conv_merged(F.silu(self.group_norm_merged(merged))) + self.skip(residue)

In [32]:
class ApplyLayer(nn.Sequential):
    
    def forward(self, x: torch.Tensor, time: torch.Tensor, context: torch.Tensor) -> torch.Tensor:
        for layer in self:
            if isinstance(layer, UnetAttentionBlock):
                x = layer(x, context)
            elif isinstance(layer, UnetResidualBlock):
                x = layer(x, time)
            else:
                x = layer(x)
        return x


In [33]:
def make_encoder_layers():
    return nn.ModuleList([
        ApplyLayer(nn.Conv2d(in_channels=4, out_channels=320, kernel_size=3, padding=1)), # Bx4x64x64 -> Bx320x64x64
        ApplyLayer(UnetResidualBlock(in_channels=320, out_channels=320), UnetAttentionBlock(n_heads=8, n_embed=40)), # Bx320x64x64 -> Bx320x64x64
        ApplyLayer(UnetResidualBlock(in_channels=320, out_channels=320), UnetAttentionBlock(n_heads=8, n_embed=40)), # Bx320x64x64 -> Bx320x64x64
        
        ApplyLayer(nn.Conv2d(in_channels=320, out_channels=320, kernel_size=3, padding=1, stride=2)), # Bx320x64x64 -> Bx320x32x32
        ApplyLayer(UnetResidualBlock(in_channels=320, out_channels=640), UnetAttentionBlock(n_heads=8, n_embed=80)), # Bx320x32x32 -> Bx320x32x32
        ApplyLayer(UnetResidualBlock(in_channels=640, out_channels=640), UnetAttentionBlock(n_heads=8, n_embed=80)), # Bx320x32x32 -> Bx320x32x32
        
        ApplyLayer(nn.Conv2d(in_channels=640, out_channels=640, kernel_size=3, padding=1, stride=2)), # Bx640x32x32 -> Bx640x16x16
        ApplyLayer(UnetResidualBlock(in_channels=640, out_channels=1280), UnetAttentionBlock(n_heads=8, n_embed=160)), # Bx640x16x16 -> Bx640x16x16
        ApplyLayer(UnetResidualBlock(in_channels=1280, out_channels=1280), UnetAttentionBlock(n_heads=8, n_embed=160)), # Bx640x16x16 -> Bx640x16x16
        
        ApplyLayer(nn.Conv2d(in_channels=1280, out_channels=1280, kernel_size=3, padding=1, stride=2)), # Bx1280x16x16 -> Bx1280x8x8
        ApplyLayer(UnetResidualBlock(in_channels=1280, out_channels=1280)),
        ApplyLayer(UnetResidualBlock(in_channels=1280, out_channels=1280)),
    ])

In [34]:
def make_decoder_layers():
    return nn.ModuleList([
        ApplyLayer(UnetResidualBlock(in_channels=2560, out_channels=1280)), # Bx2560x8x8 -> Bx1280x8x8
        ApplyLayer(UnetResidualBlock(in_channels=2560, out_channels=1280)), # Bx2560x8x8 -> Bx1280x8x8
        
        ApplyLayer(UnetResidualBlock(in_channels=2560, out_channels=1280), Upsample(in_channels=1280)), # Bx2560x8x8 -> Bx1280x16x16
        ApplyLayer(UnetResidualBlock(in_channels=2560, out_channels=1280), UnetAttentionBlock(n_heads=8, n_embed=160)), # Bx2560x16x16 -> Bx1280x16x16
        ApplyLayer(UnetResidualBlock(in_channels=2560, out_channels=1280), UnetAttentionBlock(n_heads=8, n_embed=160)), # Bx2560x16x16 -> Bx1280x16x16
        
        ApplyLayer(UnetResidualBlock(in_channels=1920, out_channels=1280), UnetAttentionBlock(n_heads=8, n_embed=160), Upsample(in_channels=1280)), # Bx2560x16x16 -> Bx1280x32x32
        ApplyLayer(UnetResidualBlock(in_channels=1920, out_channels=640), UnetAttentionBlock(n_heads=8, n_embed=80)), # Bx1920x32x32 -> Bx640x32x32
        ApplyLayer(UnetResidualBlock(in_channels=1280, out_channels=640), UnetAttentionBlock(n_heads=8, n_embed=80)), # Bx1280x32x32 -> Bx640x32x32
        
        ApplyLayer(UnetResidualBlock(in_channels=960, out_channels=640), UnetAttentionBlock(n_heads=8, n_embed=80), Upsample(in_channels=640)), # Bx960x32x32 -> Bx640x64x64
        ApplyLayer(UnetResidualBlock(in_channels=960, out_channels=320), UnetAttentionBlock(n_heads=8, n_embed=40)), # Bx960x32x32 -> Bx320x64x64
        ApplyLayer(UnetResidualBlock(in_channels=640, out_channels=320), UnetAttentionBlock(n_heads=8, n_embed=40)), # Bx640x32x32 -> Bx320x64x64
        ApplyLayer(UnetResidualBlock(in_channels=640, out_channels=320), UnetAttentionBlock(n_heads=8, n_embed=40)), # Bx640x32x32 -> Bx320x64x64  
    ])

In [35]:
class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = self._make_encoder_layers()
        self.bottle_neck = ApplyLayer(
            UnetResidualBlock(in_channels=1280, out_channels=1280),
            UnetAttentionBlock(n_heads=8, n_embed=160),
            UnetResidualBlock(in_channels=1280, out_channels=1280)
        )
        self.decoder   = self._make_decoder_layers()
        
    def forward(self, x: torch.Tensor, time: torch.Tensor, context: torch.Tensor) -> torch.Tensor:
        # x: Bx4x64x64
        # time: 1x1280
        # context: Bx77x768
        skip_connections = []
        for enc_layer in self.encoder:
            x = enc_layer(x, time, context)
            skip_connections.append(x)
            
        x = self.bottle_neck(skip_connections[-1], time, context)
        
        for dec_layer in self.decoder:
            x = torch.cat([x, skip_connections.pop()], dim=1)
            x = dec_layer(x, time, context)
        
        return x # Bx320x64x64
    
    
    def _make_encoder_layers(self):
        return nn.ModuleList([
            ApplyLayer(nn.Conv2d(in_channels=4, out_channels=320, kernel_size=3, padding=1)), # Bx4x64x64 -> Bx320x64x64
            ApplyLayer(UnetResidualBlock(in_channels=320, out_channels=320), UnetAttentionBlock(n_heads=8, n_embed=40)), # Bx320x64x64 -> Bx320x64x64
            ApplyLayer(UnetResidualBlock(in_channels=320, out_channels=320), UnetAttentionBlock(n_heads=8, n_embed=40)), # Bx320x64x64 -> Bx320x64x64
            
            ApplyLayer(nn.Conv2d(in_channels=320, out_channels=320, kernel_size=3, padding=1, stride=2)), # Bx320x64x64 -> Bx320x32x32
            ApplyLayer(UnetResidualBlock(in_channels=320, out_channels=640), UnetAttentionBlock(n_heads=8, n_embed=80)), # Bx320x32x32 -> Bx320x32x32
            ApplyLayer(UnetResidualBlock(in_channels=640, out_channels=640), UnetAttentionBlock(n_heads=8, n_embed=80)), # Bx320x32x32 -> Bx320x32x32
            
            ApplyLayer(nn.Conv2d(in_channels=640, out_channels=640, kernel_size=3, padding=1, stride=2)), # Bx640x32x32 -> Bx640x16x16
            ApplyLayer(UnetResidualBlock(in_channels=640, out_channels=1280), UnetAttentionBlock(n_heads=8, n_embed=160)), # Bx640x16x16 -> Bx640x16x16
            ApplyLayer(UnetResidualBlock(in_channels=1280, out_channels=1280), UnetAttentionBlock(n_heads=8, n_embed=160)), # Bx640x16x16 -> Bx640x16x16
            
            ApplyLayer(nn.Conv2d(in_channels=1280, out_channels=1280, kernel_size=3, padding=1, stride=2)), # Bx1280x16x16 -> Bx1280x8x8
            ApplyLayer(UnetResidualBlock(in_channels=1280, out_channels=1280)),
            ApplyLayer(UnetResidualBlock(in_channels=1280, out_channels=1280)),
        ])
        
    def _make_decoder_layers(self):
        return nn.ModuleList([
            ApplyLayer(UnetResidualBlock(in_channels=2560, out_channels=1280)), # Bx2560x8x8 -> Bx1280x8x8
            ApplyLayer(UnetResidualBlock(in_channels=2560, out_channels=1280)), # Bx2560x8x8 -> Bx1280x8x8
            
            ApplyLayer(UnetResidualBlock(in_channels=2560, out_channels=1280), Upsample(in_channels=1280)), # Bx2560x8x8 -> Bx1280x16x16
            ApplyLayer(UnetResidualBlock(in_channels=2560, out_channels=1280), UnetAttentionBlock(n_heads=8, n_embed=160)), # Bx2560x16x16 -> Bx1280x16x16
            ApplyLayer(UnetResidualBlock(in_channels=2560, out_channels=1280), UnetAttentionBlock(n_heads=8, n_embed=160)), # Bx2560x16x16 -> Bx1280x16x16
            
            ApplyLayer(UnetResidualBlock(in_channels=1920, out_channels=1280), UnetAttentionBlock(n_heads=8, n_embed=160), Upsample(in_channels=1280)), # Bx2560x16x16 -> Bx1280x32x32
            ApplyLayer(UnetResidualBlock(in_channels=1920, out_channels=640), UnetAttentionBlock(n_heads=8, n_embed=80)), # Bx1920x32x32 -> Bx640x32x32
            ApplyLayer(UnetResidualBlock(in_channels=1280, out_channels=640), UnetAttentionBlock(n_heads=8, n_embed=80)), # Bx1280x32x32 -> Bx640x32x32
            
            ApplyLayer(UnetResidualBlock(in_channels=960, out_channels=640), UnetAttentionBlock(n_heads=8, n_embed=80), Upsample(in_channels=640)), # Bx960x32x32 -> Bx640x64x64
            ApplyLayer(UnetResidualBlock(in_channels=960, out_channels=320), UnetAttentionBlock(n_heads=8, n_embed=40)), # Bx960x32x32 -> Bx320x64x64
            ApplyLayer(UnetResidualBlock(in_channels=640, out_channels=320), UnetAttentionBlock(n_heads=8, n_embed=40)), # Bx640x32x32 -> Bx320x64x64
            ApplyLayer(UnetResidualBlock(in_channels=640, out_channels=320), UnetAttentionBlock(n_heads=8, n_embed=40)), # Bx640x32x32 -> Bx320x64x64  
        ])

In [36]:
class DiffusionModel(nn.Module):
    """
    A PyTorch implementation of the Diffusion Model.
    """
    
    def __init__(self) -> None:
        super().__init__()
        
        self.time_embeddings = TimeEmbeddings(time_embedding_dim=320)
        self.unet = UNet()
        self.head = Head(in_channels=320, out_channels=4)
        
    def forward(self, latents: torch.Tensor, time: torch.Tensor, context: torch.Tensor) -> torch.Tensor:
        # latents: (batch_size, latent_dim, height, width) Bx4x64x64
        # time: (1, 320)
        # context: (batch_size, seq_len, hidden_size)
        
        time_embed = self.time_embeddings(time) # (1, 320) -> (1, 1280)
        unet_out = self.unet(latents, time_embed, context) # (B, 4, 64, 64), (B, 320, 64, 64)
        return self.head(unet_out) # (B, 320, 64, 64) -> (B, 4, 64, 64)

In [37]:
x = torch.randn(1, 4, 64, 64).to('mps')
t = torch.randn(1, 320).to('mps')
context = torch.randn(1, 77, 768).to('mps')
model = DiffusionModel().to('mps')

# pred = model(x, t, context)

def print_parameters(model):
    for name, param in model.state_dict().items():
        print(name)
        


In [46]:
print_parameters(model)

time_embeddings.linear_1.weight
time_embeddings.linear_1.bias
time_embeddings.linear_2.weight
time_embeddings.linear_2.bias
unet.encoder.0.0.weight
unet.encoder.0.0.bias
unet.encoder.1.0.group_norm.weight
unet.encoder.1.0.group_norm.bias
unet.encoder.1.0.conv.weight
unet.encoder.1.0.conv.bias
unet.encoder.1.0.linear.weight
unet.encoder.1.0.linear.bias
unet.encoder.1.0.group_norm_merged.weight
unet.encoder.1.0.group_norm_merged.bias
unet.encoder.1.0.conv_merged.weight
unet.encoder.1.0.conv_merged.bias
unet.encoder.1.1.group_norm.weight
unet.encoder.1.1.group_norm.bias
unet.encoder.1.1.conv_in.weight
unet.encoder.1.1.conv_in.bias
unet.encoder.1.1.attention_1.QKV.weight
unet.encoder.1.1.attention_1.O.weight
unet.encoder.1.1.attention_1.O.bias
unet.encoder.1.1.linear_1.weight
unet.encoder.1.1.linear_1.bias
unet.encoder.1.1.linear_2.weight
unet.encoder.1.1.linear_2.bias
unet.encoder.1.1.attention_2.Q.weight
unet.encoder.1.1.attention_2.K.weight
unet.encoder.1.1.attention_2.V.weight
unet.enc

In [38]:
input_file = '../data/checkpoints/v1-5-pruned-emaonly.ckpt'
device = 'mps'
original_model = torch.load(input_file, map_location=device, weights_only = False)["state_dict"]

In [59]:
converted = {}
converted['diffusion'] = {}

In [60]:
converted['diffusion'] = {}
converted['diffusion']['time_embeddings.linear_1.weight'] = original_model['model.diffusion_model.time_embed.0.weight']
converted['diffusion']['time_embeddings.linear_1.bias'] = original_model['model.diffusion_model.time_embed.0.bias']
converted['diffusion']['time_embeddings.linear_2.weight'] = original_model['model.diffusion_model.time_embed.2.weight']
converted['diffusion']['time_embeddings.linear_2.bias'] = original_model['model.diffusion_model.time_embed.2.bias']
converted['diffusion']['unet.encoder.0.0.weight'] =  original_model['model.diffusion_model.input_blocks.0.0.weight']
converted['diffusion']['unet.encoder.0.0.bias'] =  original_model['model.diffusion_model.input_blocks.0.0.bias']
converted['diffusion']['unet.encoder.1.0.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.1.0.in_layers.0.weight']
converted['diffusion']['unet.encoder.1.0.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.1.0.in_layers.0.bias']
converted['diffusion']['unet.encoder.1.0.conv.weight'] =  original_model['model.diffusion_model.input_blocks.1.0.in_layers.2.weight']
converted['diffusion']['unet.encoder.1.0.conv.bias'] =  original_model['model.diffusion_model.input_blocks.1.0.in_layers.2.bias']
converted['diffusion']['unet.encoder.1.0.linear.weight'] =  original_model['model.diffusion_model.input_blocks.1.0.emb_layers.1.weight']
converted['diffusion']['unet.encoder.1.0.linear.bias'] =  original_model['model.diffusion_model.input_blocks.1.0.emb_layers.1.bias']
converted['diffusion']['unet.encoder.1.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.input_blocks.1.0.out_layers.0.weight']
converted['diffusion']['unet.encoder.1.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.input_blocks.1.0.out_layers.0.bias']
converted['diffusion']['unet.encoder.1.0.conv_merged.weight'] =  original_model['model.diffusion_model.input_blocks.1.0.out_layers.3.weight']
converted['diffusion']['unet.encoder.1.0.conv_merged.bias'] =  original_model['model.diffusion_model.input_blocks.1.0.out_layers.3.bias']
converted['diffusion']['unet.encoder.1.1.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.1.1.norm.weight']
converted['diffusion']['unet.encoder.1.1.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.1.1.norm.bias']
converted['diffusion']['unet.encoder.1.1.conv_in.weight'] =  original_model['model.diffusion_model.input_blocks.1.1.proj_in.weight']
converted['diffusion']['unet.encoder.1.1.conv_in.bias'] =  original_model['model.diffusion_model.input_blocks.1.1.proj_in.bias']
converted['diffusion']['unet.encoder.1.1.attention_1.QKV.weight'] =  torch.cat((original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.encoder.1.1.attention_1.O.weight'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.encoder.1.1.attention_1.O.bias'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.encoder.1.1.linear_1.weight'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.encoder.1.1.linear_1.bias'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.encoder.1.1.linear_2.weight'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.encoder.1.1.linear_2.bias'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.encoder.1.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.encoder.1.1.attention_2.K.weight'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.encoder.1.1.attention_2.V.weight'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.encoder.1.1.attention_2.O.weight'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.encoder.1.1.attention_2.O.bias'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.encoder.1.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.encoder.1.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.encoder.1.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.encoder.1.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.encoder.1.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.encoder.1.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.input_blocks.1.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.encoder.1.1.conv_out.weight'] =  original_model['model.diffusion_model.input_blocks.1.1.proj_out.weight']
converted['diffusion']['unet.encoder.1.1.conv_out.bias'] =  original_model['model.diffusion_model.input_blocks.1.1.proj_out.bias']
converted['diffusion']['unet.encoder.2.0.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.2.0.in_layers.0.weight']
converted['diffusion']['unet.encoder.2.0.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.2.0.in_layers.0.bias']
converted['diffusion']['unet.encoder.2.0.conv.weight'] =  original_model['model.diffusion_model.input_blocks.2.0.in_layers.2.weight']
converted['diffusion']['unet.encoder.2.0.conv.bias'] =  original_model['model.diffusion_model.input_blocks.2.0.in_layers.2.bias']
converted['diffusion']['unet.encoder.2.0.linear.weight'] =  original_model['model.diffusion_model.input_blocks.2.0.emb_layers.1.weight']
converted['diffusion']['unet.encoder.2.0.linear.bias'] =  original_model['model.diffusion_model.input_blocks.2.0.emb_layers.1.bias']
converted['diffusion']['unet.encoder.2.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.input_blocks.2.0.out_layers.0.weight']
converted['diffusion']['unet.encoder.2.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.input_blocks.2.0.out_layers.0.bias']
converted['diffusion']['unet.encoder.2.0.conv_merged.weight'] =  original_model['model.diffusion_model.input_blocks.2.0.out_layers.3.weight']
converted['diffusion']['unet.encoder.2.0.conv_merged.bias'] =  original_model['model.diffusion_model.input_blocks.2.0.out_layers.3.bias']
converted['diffusion']['unet.encoder.2.1.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.2.1.norm.weight']
converted['diffusion']['unet.encoder.2.1.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.2.1.norm.bias']
converted['diffusion']['unet.encoder.2.1.conv_in.weight'] =  original_model['model.diffusion_model.input_blocks.2.1.proj_in.weight']
converted['diffusion']['unet.encoder.2.1.conv_in.bias'] =  original_model['model.diffusion_model.input_blocks.2.1.proj_in.bias']
converted['diffusion']['unet.encoder.2.1.attention_1.QKV.weight'] =  torch.cat((original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.encoder.2.1.attention_1.O.weight'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.encoder.2.1.attention_1.O.bias'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.encoder.2.1.linear_1.weight'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.encoder.2.1.linear_1.bias'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.encoder.2.1.linear_2.weight'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.encoder.2.1.linear_2.bias'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.encoder.2.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.encoder.2.1.attention_2.K.weight'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.encoder.2.1.attention_2.V.weight'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.encoder.2.1.attention_2.O.weight'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.encoder.2.1.attention_2.O.bias'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.encoder.2.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.encoder.2.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.encoder.2.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.encoder.2.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.encoder.2.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.encoder.2.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.input_blocks.2.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.encoder.2.1.conv_out.weight'] =  original_model['model.diffusion_model.input_blocks.2.1.proj_out.weight']
converted['diffusion']['unet.encoder.2.1.conv_out.bias'] =  original_model['model.diffusion_model.input_blocks.2.1.proj_out.bias']
converted['diffusion']['unet.encoder.3.0.weight'] =  original_model['model.diffusion_model.input_blocks.3.0.op.weight']
converted['diffusion']['unet.encoder.3.0.bias'] =  original_model['model.diffusion_model.input_blocks.3.0.op.bias']
converted['diffusion']['unet.encoder.4.0.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.4.0.in_layers.0.weight']
converted['diffusion']['unet.encoder.4.0.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.4.0.in_layers.0.bias']
converted['diffusion']['unet.encoder.4.0.conv.weight'] =  original_model['model.diffusion_model.input_blocks.4.0.in_layers.2.weight']
converted['diffusion']['unet.encoder.4.0.conv.bias'] =  original_model['model.diffusion_model.input_blocks.4.0.in_layers.2.bias']
converted['diffusion']['unet.encoder.4.0.linear.weight'] =  original_model['model.diffusion_model.input_blocks.4.0.emb_layers.1.weight']
converted['diffusion']['unet.encoder.4.0.linear.bias'] =  original_model['model.diffusion_model.input_blocks.4.0.emb_layers.1.bias']
converted['diffusion']['unet.encoder.4.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.input_blocks.4.0.out_layers.0.weight']
converted['diffusion']['unet.encoder.4.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.input_blocks.4.0.out_layers.0.bias']
converted['diffusion']['unet.encoder.4.0.conv_merged.weight'] =  original_model['model.diffusion_model.input_blocks.4.0.out_layers.3.weight']
converted['diffusion']['unet.encoder.4.0.conv_merged.bias'] =  original_model['model.diffusion_model.input_blocks.4.0.out_layers.3.bias']
converted['diffusion']['unet.encoder.4.0.skip.weight'] =  original_model['model.diffusion_model.input_blocks.4.0.skip_connection.weight']
converted['diffusion']['unet.encoder.4.0.skip.bias'] =  original_model['model.diffusion_model.input_blocks.4.0.skip_connection.bias']
converted['diffusion']['unet.encoder.4.1.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.4.1.norm.weight']
converted['diffusion']['unet.encoder.4.1.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.4.1.norm.bias']
converted['diffusion']['unet.encoder.4.1.conv_in.weight'] =  original_model['model.diffusion_model.input_blocks.4.1.proj_in.weight']
converted['diffusion']['unet.encoder.4.1.conv_in.bias'] =  original_model['model.diffusion_model.input_blocks.4.1.proj_in.bias']
converted['diffusion']['unet.encoder.4.1.attention_1.QKV.weight'] =  torch.cat((original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.encoder.4.1.attention_1.O.weight'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.encoder.4.1.attention_1.O.bias'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.encoder.4.1.linear_1.weight'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.encoder.4.1.linear_1.bias'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.encoder.4.1.linear_2.weight'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.encoder.4.1.linear_2.bias'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.encoder.4.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.encoder.4.1.attention_2.K.weight'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.encoder.4.1.attention_2.V.weight'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.encoder.4.1.attention_2.O.weight'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.encoder.4.1.attention_2.O.bias'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.encoder.4.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.encoder.4.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.encoder.4.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.encoder.4.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.encoder.4.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.encoder.4.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.input_blocks.4.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.encoder.4.1.conv_out.weight'] =  original_model['model.diffusion_model.input_blocks.4.1.proj_out.weight']
converted['diffusion']['unet.encoder.4.1.conv_out.bias'] =  original_model['model.diffusion_model.input_blocks.4.1.proj_out.bias']
converted['diffusion']['unet.encoder.5.0.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.5.0.in_layers.0.weight']
converted['diffusion']['unet.encoder.5.0.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.5.0.in_layers.0.bias']
converted['diffusion']['unet.encoder.5.0.conv.weight'] =  original_model['model.diffusion_model.input_blocks.5.0.in_layers.2.weight']
converted['diffusion']['unet.encoder.5.0.conv.bias'] =  original_model['model.diffusion_model.input_blocks.5.0.in_layers.2.bias']
converted['diffusion']['unet.encoder.5.0.linear.weight'] =  original_model['model.diffusion_model.input_blocks.5.0.emb_layers.1.weight']
converted['diffusion']['unet.encoder.5.0.linear.bias'] =  original_model['model.diffusion_model.input_blocks.5.0.emb_layers.1.bias']
converted['diffusion']['unet.encoder.5.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.input_blocks.5.0.out_layers.0.weight']
converted['diffusion']['unet.encoder.5.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.input_blocks.5.0.out_layers.0.bias']
converted['diffusion']['unet.encoder.5.0.conv_merged.weight'] =  original_model['model.diffusion_model.input_blocks.5.0.out_layers.3.weight']
converted['diffusion']['unet.encoder.5.0.conv_merged.bias'] =  original_model['model.diffusion_model.input_blocks.5.0.out_layers.3.bias']
converted['diffusion']['unet.encoder.5.1.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.5.1.norm.weight']
converted['diffusion']['unet.encoder.5.1.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.5.1.norm.bias']
converted['diffusion']['unet.encoder.5.1.conv_in.weight'] =  original_model['model.diffusion_model.input_blocks.5.1.proj_in.weight']
converted['diffusion']['unet.encoder.5.1.conv_in.bias'] =  original_model['model.diffusion_model.input_blocks.5.1.proj_in.bias']
converted['diffusion']['unet.encoder.5.1.attention_1.QKV.weight'] =  torch.cat((original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.encoder.5.1.attention_1.O.weight'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.encoder.5.1.attention_1.O.bias'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.encoder.5.1.linear_1.weight'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.encoder.5.1.linear_1.bias'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.encoder.5.1.linear_2.weight'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.encoder.5.1.linear_2.bias'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.encoder.5.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.encoder.5.1.attention_2.K.weight'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.encoder.5.1.attention_2.V.weight'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.encoder.5.1.attention_2.O.weight'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.encoder.5.1.attention_2.O.bias'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.encoder.5.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.encoder.5.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.encoder.5.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.encoder.5.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.encoder.5.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.encoder.5.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.input_blocks.5.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.encoder.5.1.conv_out.weight'] =  original_model['model.diffusion_model.input_blocks.5.1.proj_out.weight']
converted['diffusion']['unet.encoder.5.1.conv_out.bias'] =  original_model['model.diffusion_model.input_blocks.5.1.proj_out.bias']
converted['diffusion']['unet.encoder.6.0.weight'] =  original_model['model.diffusion_model.input_blocks.6.0.op.weight']
converted['diffusion']['unet.encoder.6.0.bias'] =  original_model['model.diffusion_model.input_blocks.6.0.op.bias']
converted['diffusion']['unet.encoder.7.0.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.7.0.in_layers.0.weight']
converted['diffusion']['unet.encoder.7.0.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.7.0.in_layers.0.bias']
converted['diffusion']['unet.encoder.7.0.conv.weight'] =  original_model['model.diffusion_model.input_blocks.7.0.in_layers.2.weight']
converted['diffusion']['unet.encoder.7.0.conv.bias'] =  original_model['model.diffusion_model.input_blocks.7.0.in_layers.2.bias']
converted['diffusion']['unet.encoder.7.0.linear.weight'] =  original_model['model.diffusion_model.input_blocks.7.0.emb_layers.1.weight']
converted['diffusion']['unet.encoder.7.0.linear.bias'] =  original_model['model.diffusion_model.input_blocks.7.0.emb_layers.1.bias']
converted['diffusion']['unet.encoder.7.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.input_blocks.7.0.out_layers.0.weight']
converted['diffusion']['unet.encoder.7.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.input_blocks.7.0.out_layers.0.bias']
converted['diffusion']['unet.encoder.7.0.conv_merged.weight'] =  original_model['model.diffusion_model.input_blocks.7.0.out_layers.3.weight']
converted['diffusion']['unet.encoder.7.0.conv_merged.bias'] =  original_model['model.diffusion_model.input_blocks.7.0.out_layers.3.bias']
converted['diffusion']['unet.encoder.7.0.skip.weight'] =  original_model['model.diffusion_model.input_blocks.7.0.skip_connection.weight']
converted['diffusion']['unet.encoder.7.0.skip.bias'] =  original_model['model.diffusion_model.input_blocks.7.0.skip_connection.bias']
converted['diffusion']['unet.encoder.7.1.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.7.1.norm.weight']
converted['diffusion']['unet.encoder.7.1.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.7.1.norm.bias']
converted['diffusion']['unet.encoder.7.1.conv_in.weight'] =  original_model['model.diffusion_model.input_blocks.7.1.proj_in.weight']
converted['diffusion']['unet.encoder.7.1.conv_in.bias'] =  original_model['model.diffusion_model.input_blocks.7.1.proj_in.bias']
converted['diffusion']['unet.encoder.7.1.attention_1.O.weight'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.encoder.7.1.attention_1.O.bias'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.encoder.7.1.linear_1.weight'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.encoder.7.1.linear_1.bias'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.encoder.7.1.linear_2.weight'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.encoder.7.1.linear_2.bias'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.encoder.7.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.encoder.7.1.attention_2.K.weight'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.encoder.7.1.attention_2.V.weight'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.encoder.7.1.attention_2.O.weight'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.encoder.7.1.attention_2.O.bias'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.encoder.7.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.encoder.7.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.encoder.7.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.encoder.7.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.encoder.7.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.encoder.7.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.encoder.7.1.conv_out.weight'] =  original_model['model.diffusion_model.input_blocks.7.1.proj_out.weight']
converted['diffusion']['unet.encoder.7.1.conv_out.bias'] =  original_model['model.diffusion_model.input_blocks.7.1.proj_out.bias']
converted['diffusion']['unet.encoder.7.1.attention_1.QKV.weight'] = torch.cat((original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.input_blocks.7.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.encoder.8.0.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.8.0.in_layers.0.weight']
converted['diffusion']['unet.encoder.8.0.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.8.0.in_layers.0.bias']
converted['diffusion']['unet.encoder.8.0.conv.weight'] =  original_model['model.diffusion_model.input_blocks.8.0.in_layers.2.weight']
converted['diffusion']['unet.encoder.8.0.conv.bias'] =  original_model['model.diffusion_model.input_blocks.8.0.in_layers.2.bias']
converted['diffusion']['unet.encoder.8.0.linear.weight'] =  original_model['model.diffusion_model.input_blocks.8.0.emb_layers.1.weight']
converted['diffusion']['unet.encoder.8.0.linear.bias'] =  original_model['model.diffusion_model.input_blocks.8.0.emb_layers.1.bias']
converted['diffusion']['unet.encoder.8.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.input_blocks.8.0.out_layers.0.weight']
converted['diffusion']['unet.encoder.8.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.input_blocks.8.0.out_layers.0.bias']
converted['diffusion']['unet.encoder.8.0.conv_merged.weight'] =  original_model['model.diffusion_model.input_blocks.8.0.out_layers.3.weight']
converted['diffusion']['unet.encoder.8.0.conv_merged.bias'] =  original_model['model.diffusion_model.input_blocks.8.0.out_layers.3.bias']
converted['diffusion']['unet.encoder.8.1.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.8.1.norm.weight']
converted['diffusion']['unet.encoder.8.1.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.8.1.norm.bias']
converted['diffusion']['unet.encoder.8.1.conv_in.weight'] =  original_model['model.diffusion_model.input_blocks.8.1.proj_in.weight']
converted['diffusion']['unet.encoder.8.1.conv_in.bias'] =  original_model['model.diffusion_model.input_blocks.8.1.proj_in.bias']
converted['diffusion']['unet.encoder.8.1.attention_1.QKV.weight'] =  torch.cat((original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.encoder.8.1.attention_1.O.weight'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.encoder.8.1.attention_1.O.bias'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.encoder.8.1.linear_1.weight'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.encoder.8.1.linear_1.bias'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.encoder.8.1.linear_2.weight'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.encoder.8.1.linear_2.bias'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.encoder.8.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.encoder.8.1.attention_2.K.weight'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.encoder.8.1.attention_2.V.weight'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.encoder.8.1.attention_2.O.weight'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.encoder.8.1.attention_2.O.bias'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.encoder.8.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.encoder.8.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.encoder.8.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.encoder.8.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.encoder.8.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.encoder.8.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.input_blocks.8.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.encoder.8.1.conv_out.weight'] =  original_model['model.diffusion_model.input_blocks.8.1.proj_out.weight']
converted['diffusion']['unet.encoder.8.1.conv_out.bias'] =  original_model['model.diffusion_model.input_blocks.8.1.proj_out.bias']
converted['diffusion']['unet.encoder.9.0.weight'] =  original_model['model.diffusion_model.input_blocks.9.0.op.weight']
converted['diffusion']['unet.encoder.9.0.bias'] =  original_model['model.diffusion_model.input_blocks.9.0.op.bias']
converted['diffusion']['unet.encoder.10.0.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.10.0.in_layers.0.weight']
converted['diffusion']['unet.encoder.10.0.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.10.0.in_layers.0.bias']
converted['diffusion']['unet.encoder.10.0.conv.weight'] =  original_model['model.diffusion_model.input_blocks.10.0.in_layers.2.weight']
converted['diffusion']['unet.encoder.10.0.conv.bias'] =  original_model['model.diffusion_model.input_blocks.10.0.in_layers.2.bias']
converted['diffusion']['unet.encoder.10.0.linear.weight'] =  original_model['model.diffusion_model.input_blocks.10.0.emb_layers.1.weight']
converted['diffusion']['unet.encoder.10.0.linear.bias'] =  original_model['model.diffusion_model.input_blocks.10.0.emb_layers.1.bias']
converted['diffusion']['unet.encoder.10.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.input_blocks.10.0.out_layers.0.weight']
converted['diffusion']['unet.encoder.10.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.input_blocks.10.0.out_layers.0.bias']
converted['diffusion']['unet.encoder.10.0.conv_merged.weight'] =  original_model['model.diffusion_model.input_blocks.10.0.out_layers.3.weight']
converted['diffusion']['unet.encoder.10.0.conv_merged.bias'] =  original_model['model.diffusion_model.input_blocks.10.0.out_layers.3.bias']
converted['diffusion']['unet.encoder.11.0.group_norm.weight'] =  original_model['model.diffusion_model.input_blocks.11.0.in_layers.0.weight']
converted['diffusion']['unet.encoder.11.0.group_norm.bias'] =  original_model['model.diffusion_model.input_blocks.11.0.in_layers.0.bias']
converted['diffusion']['unet.encoder.11.0.conv.weight'] =  original_model['model.diffusion_model.input_blocks.11.0.in_layers.2.weight']
converted['diffusion']['unet.encoder.11.0.conv.bias'] =  original_model['model.diffusion_model.input_blocks.11.0.in_layers.2.bias']
converted['diffusion']['unet.encoder.11.0.linear.weight'] =  original_model['model.diffusion_model.input_blocks.11.0.emb_layers.1.weight']
converted['diffusion']['unet.encoder.11.0.linear.bias'] =  original_model['model.diffusion_model.input_blocks.11.0.emb_layers.1.bias']
converted['diffusion']['unet.encoder.11.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.input_blocks.11.0.out_layers.0.weight']
converted['diffusion']['unet.encoder.11.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.input_blocks.11.0.out_layers.0.bias']
converted['diffusion']['unet.encoder.11.0.conv_merged.weight'] =  original_model['model.diffusion_model.input_blocks.11.0.out_layers.3.weight']
converted['diffusion']['unet.encoder.11.0.conv_merged.bias'] =  original_model['model.diffusion_model.input_blocks.11.0.out_layers.3.bias']

converted['diffusion']['unet.bottle_neck.0.group_norm.weight'] =  original_model['model.diffusion_model.middle_block.0.in_layers.0.weight']
converted['diffusion']['unet.bottle_neck.0.group_norm.bias'] =  original_model['model.diffusion_model.middle_block.0.in_layers.0.bias']
converted['diffusion']['unet.bottle_neck.0.conv.weight'] =  original_model['model.diffusion_model.middle_block.0.in_layers.2.weight']
converted['diffusion']['unet.bottle_neck.0.conv.bias'] =  original_model['model.diffusion_model.middle_block.0.in_layers.2.bias']
converted['diffusion']['unet.bottle_neck.0.linear.weight'] =  original_model['model.diffusion_model.middle_block.0.emb_layers.1.weight']
converted['diffusion']['unet.bottle_neck.0.linear.bias'] =  original_model['model.diffusion_model.middle_block.0.emb_layers.1.bias']
converted['diffusion']['unet.bottle_neck.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.middle_block.0.out_layers.0.weight']
converted['diffusion']['unet.bottle_neck.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.middle_block.0.out_layers.0.bias']
converted['diffusion']['unet.bottle_neck.0.conv_merged.weight'] =  original_model['model.diffusion_model.middle_block.0.out_layers.3.weight']
converted['diffusion']['unet.bottle_neck.0.conv_merged.bias'] =  original_model['model.diffusion_model.middle_block.0.out_layers.3.bias']
converted['diffusion']['unet.bottle_neck.1.group_norm.weight'] =  original_model['model.diffusion_model.middle_block.1.norm.weight']
converted['diffusion']['unet.bottle_neck.1.group_norm.bias'] =  original_model['model.diffusion_model.middle_block.1.norm.bias']
converted['diffusion']['unet.bottle_neck.1.conv_in.weight'] =  original_model['model.diffusion_model.middle_block.1.proj_in.weight']
converted['diffusion']['unet.bottle_neck.1.conv_in.bias'] =  original_model['model.diffusion_model.middle_block.1.proj_in.bias']
converted['diffusion']['unet.bottle_neck.1.attention_1.O.weight'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.bottle_neck.1.attention_1.O.bias'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.bottle_neck.1.linear_1.weight'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.bottle_neck.1.linear_1.bias'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.bottle_neck.1.linear_2.weight'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.bottle_neck.1.linear_2.bias'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.bottle_neck.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.bottle_neck.1.attention_2.K.weight'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.bottle_neck.1.attention_2.V.weight'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.bottle_neck.1.attention_2.O.weight'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.bottle_neck.1.attention_2.O.bias'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.bottle_neck.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.bottle_neck.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.bottle_neck.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.bottle_neck.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.bottle_neck.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.bottle_neck.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.bottle_neck.1.conv_out.weight'] =  original_model['model.diffusion_model.middle_block.1.proj_out.weight']
converted['diffusion']['unet.bottle_neck.1.conv_out.bias'] =  original_model['model.diffusion_model.middle_block.1.proj_out.bias']
converted['diffusion']['unet.bottle_neck.2.group_norm.weight'] =  original_model['model.diffusion_model.middle_block.2.in_layers.0.weight']
converted['diffusion']['unet.bottle_neck.2.group_norm.bias'] =  original_model['model.diffusion_model.middle_block.2.in_layers.0.bias']
converted['diffusion']['unet.bottle_neck.2.conv.weight'] =  original_model['model.diffusion_model.middle_block.2.in_layers.2.weight']
converted['diffusion']['unet.bottle_neck.2.conv.bias'] =  original_model['model.diffusion_model.middle_block.2.in_layers.2.bias']
converted['diffusion']['unet.bottle_neck.2.linear.weight'] =  original_model['model.diffusion_model.middle_block.2.emb_layers.1.weight']
converted['diffusion']['unet.bottle_neck.2.linear.bias'] =  original_model['model.diffusion_model.middle_block.2.emb_layers.1.bias']
converted['diffusion']['unet.bottle_neck.2.group_norm_merged.weight'] =  original_model['model.diffusion_model.middle_block.2.out_layers.0.weight']
converted['diffusion']['unet.bottle_neck.2.group_norm_merged.bias'] =  original_model['model.diffusion_model.middle_block.2.out_layers.0.bias']
converted['diffusion']['unet.bottle_neck.2.conv_merged.weight'] =  original_model['model.diffusion_model.middle_block.2.out_layers.3.weight']
converted['diffusion']['unet.bottle_neck.2.conv_merged.bias'] =  original_model['model.diffusion_model.middle_block.2.out_layers.3.bias']
converted['diffusion']['unet.bottle_neck.1.attention_1.QKV.weight'] = torch.cat((original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.middle_block.1.transformer_blocks.0.attn1.to_v.weight']), 0)

converted['diffusion']['unet.decoder.0.0.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.0.0.in_layers.0.weight']
converted['diffusion']['unet.decoder.0.0.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.0.0.in_layers.0.bias']
converted['diffusion']['unet.decoder.0.0.conv.weight'] =  original_model['model.diffusion_model.output_blocks.0.0.in_layers.2.weight']
converted['diffusion']['unet.decoder.0.0.conv.bias'] =  original_model['model.diffusion_model.output_blocks.0.0.in_layers.2.bias']
converted['diffusion']['unet.decoder.0.0.linear.weight'] =  original_model['model.diffusion_model.output_blocks.0.0.emb_layers.1.weight']
converted['diffusion']['unet.decoder.0.0.linear.bias'] =  original_model['model.diffusion_model.output_blocks.0.0.emb_layers.1.bias']
converted['diffusion']['unet.decoder.0.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.output_blocks.0.0.out_layers.0.weight']
converted['diffusion']['unet.decoder.0.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.output_blocks.0.0.out_layers.0.bias']
converted['diffusion']['unet.decoder.0.0.conv_merged.weight'] =  original_model['model.diffusion_model.output_blocks.0.0.out_layers.3.weight']
converted['diffusion']['unet.decoder.0.0.conv_merged.bias'] =  original_model['model.diffusion_model.output_blocks.0.0.out_layers.3.bias']
converted['diffusion']['unet.decoder.0.0.skip.weight'] =  original_model['model.diffusion_model.output_blocks.0.0.skip_connection.weight']
converted['diffusion']['unet.decoder.0.0.skip.bias'] =  original_model['model.diffusion_model.output_blocks.0.0.skip_connection.bias']
converted['diffusion']['unet.decoder.1.0.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.1.0.in_layers.0.weight']
converted['diffusion']['unet.decoder.1.0.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.1.0.in_layers.0.bias']
converted['diffusion']['unet.decoder.1.0.conv.weight'] =  original_model['model.diffusion_model.output_blocks.1.0.in_layers.2.weight']
converted['diffusion']['unet.decoder.1.0.conv.bias'] =  original_model['model.diffusion_model.output_blocks.1.0.in_layers.2.bias']
converted['diffusion']['unet.decoder.1.0.linear.weight'] =  original_model['model.diffusion_model.output_blocks.1.0.emb_layers.1.weight']
converted['diffusion']['unet.decoder.1.0.linear.bias'] =  original_model['model.diffusion_model.output_blocks.1.0.emb_layers.1.bias']
converted['diffusion']['unet.decoder.1.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.output_blocks.1.0.out_layers.0.weight']
converted['diffusion']['unet.decoder.1.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.output_blocks.1.0.out_layers.0.bias']
converted['diffusion']['unet.decoder.1.0.conv_merged.weight'] =  original_model['model.diffusion_model.output_blocks.1.0.out_layers.3.weight']
converted['diffusion']['unet.decoder.1.0.conv_merged.bias'] =  original_model['model.diffusion_model.output_blocks.1.0.out_layers.3.bias']
converted['diffusion']['unet.decoder.1.0.skip.weight'] =  original_model['model.diffusion_model.output_blocks.1.0.skip_connection.weight']
converted['diffusion']['unet.decoder.1.0.skip.bias'] =  original_model['model.diffusion_model.output_blocks.1.0.skip_connection.bias']
converted['diffusion']['unet.decoder.2.0.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.2.0.in_layers.0.weight']
converted['diffusion']['unet.decoder.2.0.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.2.0.in_layers.0.bias']
converted['diffusion']['unet.decoder.2.0.conv.weight'] =  original_model['model.diffusion_model.output_blocks.2.0.in_layers.2.weight']
converted['diffusion']['unet.decoder.2.0.conv.bias'] =  original_model['model.diffusion_model.output_blocks.2.0.in_layers.2.bias']
converted['diffusion']['unet.decoder.2.0.linear.weight'] =  original_model['model.diffusion_model.output_blocks.2.0.emb_layers.1.weight']
converted['diffusion']['unet.decoder.2.0.linear.bias'] =  original_model['model.diffusion_model.output_blocks.2.0.emb_layers.1.bias']
converted['diffusion']['unet.decoder.2.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.output_blocks.2.0.out_layers.0.weight']
converted['diffusion']['unet.decoder.2.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.output_blocks.2.0.out_layers.0.bias']
converted['diffusion']['unet.decoder.2.0.conv_merged.weight'] =  original_model['model.diffusion_model.output_blocks.2.0.out_layers.3.weight']
converted['diffusion']['unet.decoder.2.0.conv_merged.bias'] =  original_model['model.diffusion_model.output_blocks.2.0.out_layers.3.bias']
converted['diffusion']['unet.decoder.2.0.skip.weight'] =  original_model['model.diffusion_model.output_blocks.2.0.skip_connection.weight']
converted['diffusion']['unet.decoder.2.0.skip.bias'] =  original_model['model.diffusion_model.output_blocks.2.0.skip_connection.bias']
converted['diffusion']['unet.decoder.2.1.conv.weight'] =  original_model['model.diffusion_model.output_blocks.2.1.conv.weight']
converted['diffusion']['unet.decoder.2.1.conv.bias'] =  original_model['model.diffusion_model.output_blocks.2.1.conv.bias']
converted['diffusion']['unet.decoder.3.0.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.3.0.in_layers.0.weight']
converted['diffusion']['unet.decoder.3.0.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.3.0.in_layers.0.bias']
converted['diffusion']['unet.decoder.3.0.conv.weight'] =  original_model['model.diffusion_model.output_blocks.3.0.in_layers.2.weight']
converted['diffusion']['unet.decoder.3.0.conv.bias'] =  original_model['model.diffusion_model.output_blocks.3.0.in_layers.2.bias']
converted['diffusion']['unet.decoder.3.0.linear.weight'] =  original_model['model.diffusion_model.output_blocks.3.0.emb_layers.1.weight']
converted['diffusion']['unet.decoder.3.0.linear.bias'] =  original_model['model.diffusion_model.output_blocks.3.0.emb_layers.1.bias']
converted['diffusion']['unet.decoder.3.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.output_blocks.3.0.out_layers.0.weight']
converted['diffusion']['unet.decoder.3.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.output_blocks.3.0.out_layers.0.bias']
converted['diffusion']['unet.decoder.3.0.conv_merged.weight'] =  original_model['model.diffusion_model.output_blocks.3.0.out_layers.3.weight']
converted['diffusion']['unet.decoder.3.0.conv_merged.bias'] =  original_model['model.diffusion_model.output_blocks.3.0.out_layers.3.bias']
converted['diffusion']['unet.decoder.3.0.skip.weight'] =  original_model['model.diffusion_model.output_blocks.3.0.skip_connection.weight']
converted['diffusion']['unet.decoder.3.0.skip.bias'] =  original_model['model.diffusion_model.output_blocks.3.0.skip_connection.bias']
converted['diffusion']['unet.decoder.3.1.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.3.1.norm.weight']
converted['diffusion']['unet.decoder.3.1.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.3.1.norm.bias']
converted['diffusion']['unet.decoder.3.1.conv_in.weight'] =  original_model['model.diffusion_model.output_blocks.3.1.proj_in.weight']
converted['diffusion']['unet.decoder.3.1.conv_in.bias'] =  original_model['model.diffusion_model.output_blocks.3.1.proj_in.bias']
converted['diffusion']['unet.decoder.3.1.attention_1.QKV.weight'] = torch.cat((original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.decoder.3.1.attention_1.O.weight'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.decoder.3.1.attention_1.O.bias'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.decoder.3.1.linear_1.weight'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.decoder.3.1.linear_1.bias'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.decoder.3.1.linear_2.weight'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.decoder.3.1.linear_2.bias'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.decoder.3.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.decoder.3.1.attention_2.K.weight'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.decoder.3.1.attention_2.V.weight'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.decoder.3.1.attention_2.O.weight'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.decoder.3.1.attention_2.O.bias'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.decoder.3.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.decoder.3.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.decoder.3.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.decoder.3.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.decoder.3.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.decoder.3.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.decoder.3.1.conv_out.weight'] =  original_model['model.diffusion_model.output_blocks.3.1.proj_out.weight']
converted['diffusion']['unet.decoder.3.1.conv_out.bias'] =  original_model['model.diffusion_model.output_blocks.3.1.proj_out.bias']

converted['diffusion']['unet.decoder.4.0.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.4.0.in_layers.0.weight']
converted['diffusion']['unet.decoder.4.0.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.4.0.in_layers.0.bias']
converted['diffusion']['unet.decoder.4.0.conv.weight'] =  original_model['model.diffusion_model.output_blocks.4.0.in_layers.2.weight']
converted['diffusion']['unet.decoder.4.0.conv.bias'] =  original_model['model.diffusion_model.output_blocks.4.0.in_layers.2.bias']
converted['diffusion']['unet.decoder.4.0.linear.weight'] =  original_model['model.diffusion_model.output_blocks.4.0.emb_layers.1.weight']
converted['diffusion']['unet.decoder.4.0.linear.bias'] =  original_model['model.diffusion_model.output_blocks.4.0.emb_layers.1.bias']
converted['diffusion']['unet.decoder.4.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.output_blocks.4.0.out_layers.0.weight']
converted['diffusion']['unet.decoder.4.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.output_blocks.4.0.out_layers.0.bias']
converted['diffusion']['unet.decoder.4.0.conv_merged.weight'] =  original_model['model.diffusion_model.output_blocks.4.0.out_layers.3.weight']
converted['diffusion']['unet.decoder.4.0.conv_merged.bias'] =  original_model['model.diffusion_model.output_blocks.4.0.out_layers.3.bias']
converted['diffusion']['unet.decoder.4.0.skip.weight'] =  original_model['model.diffusion_model.output_blocks.4.0.skip_connection.weight']
converted['diffusion']['unet.decoder.4.0.skip.bias'] =  original_model['model.diffusion_model.output_blocks.4.0.skip_connection.bias']
converted['diffusion']['unet.decoder.4.1.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.4.1.norm.weight']
converted['diffusion']['unet.decoder.4.1.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.4.1.norm.bias']
converted['diffusion']['unet.decoder.4.1.conv_in.weight'] =  original_model['model.diffusion_model.output_blocks.4.1.proj_in.weight']
converted['diffusion']['unet.decoder.4.1.conv_in.bias'] =  original_model['model.diffusion_model.output_blocks.4.1.proj_in.bias']
converted['diffusion']['unet.decoder.4.1.attention_1.O.weight'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.decoder.4.1.attention_1.O.bias'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.decoder.4.1.linear_1.weight'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.decoder.4.1.linear_1.bias'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.decoder.4.1.linear_2.weight'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.decoder.4.1.linear_2.bias'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.decoder.4.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.decoder.4.1.attention_2.K.weight'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.decoder.4.1.attention_2.V.weight'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.decoder.4.1.attention_2.O.weight'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.decoder.4.1.attention_2.O.bias'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.decoder.4.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.decoder.4.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.decoder.4.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.decoder.4.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.decoder.4.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.decoder.4.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.decoder.4.1.conv_out.weight'] =  original_model['model.diffusion_model.output_blocks.4.1.proj_out.weight']
converted['diffusion']['unet.decoder.4.1.conv_out.bias'] =  original_model['model.diffusion_model.output_blocks.4.1.proj_out.bias']
converted['diffusion']['unet.decoder.5.0.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.5.0.in_layers.0.weight']
converted['diffusion']['unet.decoder.5.0.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.5.0.in_layers.0.bias']
converted['diffusion']['unet.decoder.5.0.conv.weight'] =  original_model['model.diffusion_model.output_blocks.5.0.in_layers.2.weight']
converted['diffusion']['unet.decoder.5.0.conv.bias'] =  original_model['model.diffusion_model.output_blocks.5.0.in_layers.2.bias']
converted['diffusion']['unet.decoder.5.0.linear.weight'] =  original_model['model.diffusion_model.output_blocks.5.0.emb_layers.1.weight']
converted['diffusion']['unet.decoder.5.0.linear.bias'] =  original_model['model.diffusion_model.output_blocks.5.0.emb_layers.1.bias']
converted['diffusion']['unet.decoder.5.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.output_blocks.5.0.out_layers.0.weight']
converted['diffusion']['unet.decoder.5.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.output_blocks.5.0.out_layers.0.bias']
converted['diffusion']['unet.decoder.5.0.conv_merged.weight'] =  original_model['model.diffusion_model.output_blocks.5.0.out_layers.3.weight']
converted['diffusion']['unet.decoder.5.0.conv_merged.bias'] =  original_model['model.diffusion_model.output_blocks.5.0.out_layers.3.bias']
converted['diffusion']['unet.decoder.5.0.skip.weight'] =  original_model['model.diffusion_model.output_blocks.5.0.skip_connection.weight']
converted['diffusion']['unet.decoder.5.0.skip.bias'] =  original_model['model.diffusion_model.output_blocks.5.0.skip_connection.bias']
converted['diffusion']['unet.decoder.5.1.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.5.1.norm.weight']
converted['diffusion']['unet.decoder.5.1.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.5.1.norm.bias']
converted['diffusion']['unet.decoder.5.1.conv_in.weight'] =  original_model['model.diffusion_model.output_blocks.5.1.proj_in.weight']
converted['diffusion']['unet.decoder.5.1.conv_in.bias'] =  original_model['model.diffusion_model.output_blocks.5.1.proj_in.bias']
converted['diffusion']['unet.decoder.5.1.attention_1.O.weight'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.decoder.5.1.attention_1.O.bias'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.decoder.5.1.linear_1.weight'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.decoder.5.1.linear_1.bias'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.decoder.5.1.linear_2.weight'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.decoder.5.1.linear_2.bias'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.decoder.5.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.decoder.5.1.attention_2.K.weight'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.decoder.5.1.attention_2.V.weight'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.decoder.5.1.attention_2.O.weight'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.decoder.5.1.attention_2.O.bias'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.decoder.5.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.decoder.5.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.decoder.5.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.decoder.5.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.decoder.5.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.decoder.5.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.decoder.5.1.conv_out.weight'] =  original_model['model.diffusion_model.output_blocks.5.1.proj_out.weight']
converted['diffusion']['unet.decoder.5.1.conv_out.bias'] =  original_model['model.diffusion_model.output_blocks.5.1.proj_out.bias']
converted['diffusion']['unet.decoder.5.2.conv.weight'] =  original_model['model.diffusion_model.output_blocks.5.2.conv.weight']
converted['diffusion']['unet.decoder.5.2.conv.bias'] =  original_model['model.diffusion_model.output_blocks.5.2.conv.bias']
converted['diffusion']['unet.decoder.6.0.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.6.0.in_layers.0.weight']
converted['diffusion']['unet.decoder.6.0.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.6.0.in_layers.0.bias']
converted['diffusion']['unet.decoder.6.0.conv.weight'] =  original_model['model.diffusion_model.output_blocks.6.0.in_layers.2.weight']
converted['diffusion']['unet.decoder.6.0.conv.bias'] =  original_model['model.diffusion_model.output_blocks.6.0.in_layers.2.bias']
converted['diffusion']['unet.decoder.6.0.linear.weight'] =  original_model['model.diffusion_model.output_blocks.6.0.emb_layers.1.weight']
converted['diffusion']['unet.decoder.6.0.linear.bias'] =  original_model['model.diffusion_model.output_blocks.6.0.emb_layers.1.bias']
converted['diffusion']['unet.decoder.6.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.output_blocks.6.0.out_layers.0.weight']
converted['diffusion']['unet.decoder.6.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.output_blocks.6.0.out_layers.0.bias']
converted['diffusion']['unet.decoder.6.0.conv_merged.weight'] =  original_model['model.diffusion_model.output_blocks.6.0.out_layers.3.weight']
converted['diffusion']['unet.decoder.6.0.conv_merged.bias'] =  original_model['model.diffusion_model.output_blocks.6.0.out_layers.3.bias']
converted['diffusion']['unet.decoder.6.0.skip.weight'] =  original_model['model.diffusion_model.output_blocks.6.0.skip_connection.weight']
converted['diffusion']['unet.decoder.6.0.skip.bias'] =  original_model['model.diffusion_model.output_blocks.6.0.skip_connection.bias']
converted['diffusion']['unet.decoder.6.1.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.6.1.norm.weight']
converted['diffusion']['unet.decoder.6.1.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.6.1.norm.bias']
converted['diffusion']['unet.decoder.6.1.conv_in.weight'] =  original_model['model.diffusion_model.output_blocks.6.1.proj_in.weight']
converted['diffusion']['unet.decoder.6.1.conv_in.bias'] =  original_model['model.diffusion_model.output_blocks.6.1.proj_in.bias']
converted['diffusion']['unet.decoder.6.1.attention_1.O.weight'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.decoder.6.1.attention_1.O.bias'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.decoder.6.1.linear_1.weight'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.decoder.6.1.linear_1.bias'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.decoder.6.1.linear_2.weight'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.decoder.6.1.linear_2.bias'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.decoder.6.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.decoder.6.1.attention_2.K.weight'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.decoder.6.1.attention_2.V.weight'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.decoder.6.1.attention_2.O.weight'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.decoder.6.1.attention_2.O.bias'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.decoder.6.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.decoder.6.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.decoder.6.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.decoder.6.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.decoder.6.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.decoder.6.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.decoder.6.1.conv_out.weight'] =  original_model['model.diffusion_model.output_blocks.6.1.proj_out.weight']
converted['diffusion']['unet.decoder.6.1.conv_out.bias'] =  original_model['model.diffusion_model.output_blocks.6.1.proj_out.bias']
converted['diffusion']['unet.decoder.7.0.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.7.0.in_layers.0.weight']
converted['diffusion']['unet.decoder.7.0.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.7.0.in_layers.0.bias']
converted['diffusion']['unet.decoder.7.0.conv.weight'] =  original_model['model.diffusion_model.output_blocks.7.0.in_layers.2.weight']
converted['diffusion']['unet.decoder.7.0.conv.bias'] =  original_model['model.diffusion_model.output_blocks.7.0.in_layers.2.bias']
converted['diffusion']['unet.decoder.7.0.linear.weight'] =  original_model['model.diffusion_model.output_blocks.7.0.emb_layers.1.weight']
converted['diffusion']['unet.decoder.7.0.linear.bias'] =  original_model['model.diffusion_model.output_blocks.7.0.emb_layers.1.bias']
converted['diffusion']['unet.decoder.7.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.output_blocks.7.0.out_layers.0.weight']
converted['diffusion']['unet.decoder.7.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.output_blocks.7.0.out_layers.0.bias']
converted['diffusion']['unet.decoder.7.0.conv_merged.weight'] =  original_model['model.diffusion_model.output_blocks.7.0.out_layers.3.weight']
converted['diffusion']['unet.decoder.7.0.conv_merged.bias'] =  original_model['model.diffusion_model.output_blocks.7.0.out_layers.3.bias']
converted['diffusion']['unet.decoder.7.0.skip.weight'] =  original_model['model.diffusion_model.output_blocks.7.0.skip_connection.weight']
converted['diffusion']['unet.decoder.7.0.skip.bias'] =  original_model['model.diffusion_model.output_blocks.7.0.skip_connection.bias']
converted['diffusion']['unet.decoder.7.1.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.7.1.norm.weight']
converted['diffusion']['unet.decoder.7.1.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.7.1.norm.bias']
converted['diffusion']['unet.decoder.7.1.conv_in.weight'] =  original_model['model.diffusion_model.output_blocks.7.1.proj_in.weight']
converted['diffusion']['unet.decoder.7.1.conv_in.bias'] =  original_model['model.diffusion_model.output_blocks.7.1.proj_in.bias']
converted['diffusion']['unet.decoder.7.1.attention_1.O.weight'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.decoder.7.1.attention_1.O.bias'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.decoder.7.1.linear_1.weight'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.decoder.7.1.linear_1.bias'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.decoder.7.1.linear_2.weight'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.decoder.7.1.linear_2.bias'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.decoder.7.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.decoder.7.1.attention_2.K.weight'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.decoder.7.1.attention_2.V.weight'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.decoder.7.1.attention_2.O.weight'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.decoder.7.1.attention_2.O.bias'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.decoder.7.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.decoder.7.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.decoder.7.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.decoder.7.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.decoder.7.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.decoder.7.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.decoder.7.1.conv_out.weight'] =  original_model['model.diffusion_model.output_blocks.7.1.proj_out.weight']
converted['diffusion']['unet.decoder.7.1.conv_out.bias'] =  original_model['model.diffusion_model.output_blocks.7.1.proj_out.bias']
converted['diffusion']['unet.decoder.8.0.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.8.0.in_layers.0.weight']
converted['diffusion']['unet.decoder.8.0.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.8.0.in_layers.0.bias']
converted['diffusion']['unet.decoder.8.0.conv.weight'] =  original_model['model.diffusion_model.output_blocks.8.0.in_layers.2.weight']
converted['diffusion']['unet.decoder.8.0.conv.bias'] =  original_model['model.diffusion_model.output_blocks.8.0.in_layers.2.bias']
converted['diffusion']['unet.decoder.8.0.linear.weight'] =  original_model['model.diffusion_model.output_blocks.8.0.emb_layers.1.weight']
converted['diffusion']['unet.decoder.8.0.linear.bias'] =  original_model['model.diffusion_model.output_blocks.8.0.emb_layers.1.bias']
converted['diffusion']['unet.decoder.8.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.output_blocks.8.0.out_layers.0.weight']
converted['diffusion']['unet.decoder.8.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.output_blocks.8.0.out_layers.0.bias']
converted['diffusion']['unet.decoder.8.0.conv_merged.weight'] =  original_model['model.diffusion_model.output_blocks.8.0.out_layers.3.weight']
converted['diffusion']['unet.decoder.8.0.conv_merged.bias'] =  original_model['model.diffusion_model.output_blocks.8.0.out_layers.3.bias']
converted['diffusion']['unet.decoder.8.0.skip.weight'] =  original_model['model.diffusion_model.output_blocks.8.0.skip_connection.weight']
converted['diffusion']['unet.decoder.8.0.skip.bias'] =  original_model['model.diffusion_model.output_blocks.8.0.skip_connection.bias']
converted['diffusion']['unet.decoder.8.1.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.8.1.norm.weight']
converted['diffusion']['unet.decoder.8.1.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.8.1.norm.bias']
converted['diffusion']['unet.decoder.8.1.conv_in.weight'] =  original_model['model.diffusion_model.output_blocks.8.1.proj_in.weight']
converted['diffusion']['unet.decoder.8.1.conv_in.bias'] =  original_model['model.diffusion_model.output_blocks.8.1.proj_in.bias']
converted['diffusion']['unet.decoder.8.1.attention_1.O.weight'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.decoder.8.1.attention_1.O.bias'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.decoder.8.1.linear_1.weight'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.decoder.8.1.linear_1.bias'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.decoder.8.1.linear_2.weight'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.decoder.8.1.linear_2.bias'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.decoder.8.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.decoder.8.1.attention_2.K.weight'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.decoder.8.1.attention_2.V.weight'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.decoder.8.1.attention_2.O.weight'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.decoder.8.1.attention_2.O.bias'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.decoder.8.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.decoder.8.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.decoder.8.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.decoder.8.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.decoder.8.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.decoder.8.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.decoder.8.1.conv_out.weight'] =  original_model['model.diffusion_model.output_blocks.8.1.proj_out.weight']
converted['diffusion']['unet.decoder.8.1.conv_out.bias'] =  original_model['model.diffusion_model.output_blocks.8.1.proj_out.bias']
converted['diffusion']['unet.decoder.8.2.conv.weight'] =  original_model['model.diffusion_model.output_blocks.8.2.conv.weight']
converted['diffusion']['unet.decoder.8.2.conv.bias'] =  original_model['model.diffusion_model.output_blocks.8.2.conv.bias']
converted['diffusion']['unet.decoder.9.0.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.9.0.in_layers.0.weight']
converted['diffusion']['unet.decoder.9.0.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.9.0.in_layers.0.bias']
converted['diffusion']['unet.decoder.9.0.conv.weight'] =  original_model['model.diffusion_model.output_blocks.9.0.in_layers.2.weight']
converted['diffusion']['unet.decoder.9.0.conv.bias'] =  original_model['model.diffusion_model.output_blocks.9.0.in_layers.2.bias']
converted['diffusion']['unet.decoder.9.0.linear.weight'] =  original_model['model.diffusion_model.output_blocks.9.0.emb_layers.1.weight']
converted['diffusion']['unet.decoder.9.0.linear.bias'] =  original_model['model.diffusion_model.output_blocks.9.0.emb_layers.1.bias']
converted['diffusion']['unet.decoder.9.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.output_blocks.9.0.out_layers.0.weight']
converted['diffusion']['unet.decoder.9.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.output_blocks.9.0.out_layers.0.bias']
converted['diffusion']['unet.decoder.9.0.conv_merged.weight'] =  original_model['model.diffusion_model.output_blocks.9.0.out_layers.3.weight']
converted['diffusion']['unet.decoder.9.0.conv_merged.bias'] =  original_model['model.diffusion_model.output_blocks.9.0.out_layers.3.bias']
converted['diffusion']['unet.decoder.9.0.skip.weight'] =  original_model['model.diffusion_model.output_blocks.9.0.skip_connection.weight']
converted['diffusion']['unet.decoder.9.0.skip.bias'] =  original_model['model.diffusion_model.output_blocks.9.0.skip_connection.bias']
converted['diffusion']['unet.decoder.9.1.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.9.1.norm.weight']
converted['diffusion']['unet.decoder.9.1.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.9.1.norm.bias']
converted['diffusion']['unet.decoder.9.1.conv_in.weight'] =  original_model['model.diffusion_model.output_blocks.9.1.proj_in.weight']
converted['diffusion']['unet.decoder.9.1.conv_in.bias'] =  original_model['model.diffusion_model.output_blocks.9.1.proj_in.bias']
converted['diffusion']['unet.decoder.9.1.attention_1.O.weight'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.decoder.9.1.attention_1.O.bias'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.decoder.9.1.linear_1.weight'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.decoder.9.1.linear_1.bias'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.decoder.9.1.linear_2.weight'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.decoder.9.1.linear_2.bias'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.decoder.9.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.decoder.9.1.attention_2.K.weight'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.decoder.9.1.attention_2.V.weight'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.decoder.9.1.attention_2.O.weight'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.decoder.9.1.attention_2.O.bias'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.decoder.9.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.decoder.9.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.decoder.9.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.decoder.9.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.decoder.9.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.decoder.9.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.decoder.9.1.conv_out.weight'] =  original_model['model.diffusion_model.output_blocks.9.1.proj_out.weight']
converted['diffusion']['unet.decoder.9.1.conv_out.bias'] =  original_model['model.diffusion_model.output_blocks.9.1.proj_out.bias']
converted['diffusion']['unet.decoder.10.0.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.10.0.in_layers.0.weight']
converted['diffusion']['unet.decoder.10.0.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.10.0.in_layers.0.bias']
converted['diffusion']['unet.decoder.10.0.conv.weight'] =  original_model['model.diffusion_model.output_blocks.10.0.in_layers.2.weight']
converted['diffusion']['unet.decoder.10.0.conv.bias'] =  original_model['model.diffusion_model.output_blocks.10.0.in_layers.2.bias']
converted['diffusion']['unet.decoder.10.0.linear.weight'] =  original_model['model.diffusion_model.output_blocks.10.0.emb_layers.1.weight']
converted['diffusion']['unet.decoder.10.0.linear.bias'] =  original_model['model.diffusion_model.output_blocks.10.0.emb_layers.1.bias']
converted['diffusion']['unet.decoder.10.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.output_blocks.10.0.out_layers.0.weight']
converted['diffusion']['unet.decoder.10.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.output_blocks.10.0.out_layers.0.bias']
converted['diffusion']['unet.decoder.10.0.conv_merged.weight'] =  original_model['model.diffusion_model.output_blocks.10.0.out_layers.3.weight']
converted['diffusion']['unet.decoder.10.0.conv_merged.bias'] =  original_model['model.diffusion_model.output_blocks.10.0.out_layers.3.bias']
converted['diffusion']['unet.decoder.10.0.skip.weight'] =  original_model['model.diffusion_model.output_blocks.10.0.skip_connection.weight']
converted['diffusion']['unet.decoder.10.0.skip.bias'] =  original_model['model.diffusion_model.output_blocks.10.0.skip_connection.bias']
converted['diffusion']['unet.decoder.10.1.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.10.1.norm.weight']
converted['diffusion']['unet.decoder.10.1.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.10.1.norm.bias']
converted['diffusion']['unet.decoder.10.1.conv_in.weight'] =  original_model['model.diffusion_model.output_blocks.10.1.proj_in.weight']
converted['diffusion']['unet.decoder.10.1.conv_in.bias'] =  original_model['model.diffusion_model.output_blocks.10.1.proj_in.bias']
converted['diffusion']['unet.decoder.10.1.attention_1.O.weight'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.decoder.10.1.attention_1.O.bias'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.decoder.10.1.linear_1.weight'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.decoder.10.1.linear_1.bias'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.decoder.10.1.linear_2.weight'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.decoder.10.1.linear_2.bias'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.decoder.10.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.decoder.10.1.attention_2.K.weight'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.decoder.10.1.attention_2.V.weight'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.decoder.10.1.attention_2.O.weight'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.decoder.10.1.attention_2.O.bias'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.decoder.10.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.decoder.10.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.decoder.10.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.decoder.10.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.decoder.10.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.decoder.10.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.decoder.10.1.conv_out.weight'] =  original_model['model.diffusion_model.output_blocks.10.1.proj_out.weight']
converted['diffusion']['unet.decoder.10.1.conv_out.bias'] =  original_model['model.diffusion_model.output_blocks.10.1.proj_out.bias']
converted['diffusion']['unet.decoder.11.0.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.11.0.in_layers.0.weight']
converted['diffusion']['unet.decoder.11.0.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.11.0.in_layers.0.bias']
converted['diffusion']['unet.decoder.11.0.conv.weight'] =  original_model['model.diffusion_model.output_blocks.11.0.in_layers.2.weight']
converted['diffusion']['unet.decoder.11.0.conv.bias'] =  original_model['model.diffusion_model.output_blocks.11.0.in_layers.2.bias']
converted['diffusion']['unet.decoder.11.0.linear.weight'] =  original_model['model.diffusion_model.output_blocks.11.0.emb_layers.1.weight']
converted['diffusion']['unet.decoder.11.0.linear.bias'] =  original_model['model.diffusion_model.output_blocks.11.0.emb_layers.1.bias']
converted['diffusion']['unet.decoder.11.0.group_norm_merged.weight'] =  original_model['model.diffusion_model.output_blocks.11.0.out_layers.0.weight']
converted['diffusion']['unet.decoder.11.0.group_norm_merged.bias'] =  original_model['model.diffusion_model.output_blocks.11.0.out_layers.0.bias']
converted['diffusion']['unet.decoder.11.0.conv_merged.weight'] =  original_model['model.diffusion_model.output_blocks.11.0.out_layers.3.weight']
converted['diffusion']['unet.decoder.11.0.conv_merged.bias'] =  original_model['model.diffusion_model.output_blocks.11.0.out_layers.3.bias']
converted['diffusion']['unet.decoder.11.0.skip.weight'] =  original_model['model.diffusion_model.output_blocks.11.0.skip_connection.weight']
converted['diffusion']['unet.decoder.11.0.skip.bias'] =  original_model['model.diffusion_model.output_blocks.11.0.skip_connection.bias']
converted['diffusion']['unet.decoder.11.1.group_norm.weight'] =  original_model['model.diffusion_model.output_blocks.11.1.norm.weight']
converted['diffusion']['unet.decoder.11.1.group_norm.bias'] =  original_model['model.diffusion_model.output_blocks.11.1.norm.bias']
converted['diffusion']['unet.decoder.11.1.conv_in.weight'] =  original_model['model.diffusion_model.output_blocks.11.1.proj_in.weight']
converted['diffusion']['unet.decoder.11.1.conv_in.bias'] =  original_model['model.diffusion_model.output_blocks.11.1.proj_in.bias']
converted['diffusion']['unet.decoder.11.1.attention_1.O.weight'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.attn1.to_out.0.weight']
converted['diffusion']['unet.decoder.11.1.attention_1.O.bias'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.attn1.to_out.0.bias']
converted['diffusion']['unet.decoder.11.1.linear_1.weight'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.ff.net.0.proj.weight']
converted['diffusion']['unet.decoder.11.1.linear_1.bias'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.ff.net.0.proj.bias']
converted['diffusion']['unet.decoder.11.1.linear_2.weight'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.ff.net.2.weight']
converted['diffusion']['unet.decoder.11.1.linear_2.bias'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.ff.net.2.bias']
converted['diffusion']['unet.decoder.11.1.attention_2.Q.weight'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.attn2.to_q.weight']
converted['diffusion']['unet.decoder.11.1.attention_2.K.weight'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.attn2.to_k.weight']
converted['diffusion']['unet.decoder.11.1.attention_2.V.weight'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.attn2.to_v.weight']
converted['diffusion']['unet.decoder.11.1.attention_2.O.weight'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.attn2.to_out.0.weight']
converted['diffusion']['unet.decoder.11.1.attention_2.O.bias'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.attn2.to_out.0.bias']
converted['diffusion']['unet.decoder.11.1.layer_norm_1.weight'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.norm1.weight']
converted['diffusion']['unet.decoder.11.1.layer_norm_1.bias'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.norm1.bias']
converted['diffusion']['unet.decoder.11.1.layer_norm_2.weight'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.norm2.weight']
converted['diffusion']['unet.decoder.11.1.layer_norm_2.bias'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.norm2.bias']
converted['diffusion']['unet.decoder.11.1.layer_norm_3.weight'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.norm3.weight']
converted['diffusion']['unet.decoder.11.1.layer_norm_3.bias'] =  original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.norm3.bias']
converted['diffusion']['unet.decoder.11.1.conv_out.weight'] =  original_model['model.diffusion_model.output_blocks.11.1.proj_out.weight']
converted['diffusion']['unet.decoder.11.1.conv_out.bias'] =  original_model['model.diffusion_model.output_blocks.11.1.proj_out.bias']
converted['diffusion']['head.group_norm.weight'] =  original_model['model.diffusion_model.out.0.weight']
converted['diffusion']['head.group_norm.bias'] =  original_model['model.diffusion_model.out.0.bias']
converted['diffusion']['head.conv.weight'] =  original_model['model.diffusion_model.out.2.weight']
converted['diffusion']['head.conv.bias'] =  original_model['model.diffusion_model.out.2.bias']

converted['diffusion']['unet.decoder.3.1.attention_1.QKV.weight'] = torch.cat((original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.output_blocks.3.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.decoder.4.1.attention_1.QKV.weight'] = torch.cat((original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.output_blocks.4.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.decoder.5.1.attention_1.QKV.weight'] = torch.cat((original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.output_blocks.5.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.decoder.6.1.attention_1.QKV.weight'] = torch.cat((original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.output_blocks.6.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.decoder.7.1.attention_1.QKV.weight'] = torch.cat((original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.output_blocks.7.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.decoder.8.1.attention_1.QKV.weight'] = torch.cat((original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.output_blocks.8.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.decoder.9.1.attention_1.QKV.weight'] = torch.cat((original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.output_blocks.9.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.decoder.10.1.attention_1.QKV.weight'] = torch.cat((original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.output_blocks.10.1.transformer_blocks.0.attn1.to_v.weight']), 0)
converted['diffusion']['unet.decoder.11.1.attention_1.QKV.weight'] = torch.cat((original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.attn1.to_q.weight'], original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.attn1.to_k.weight'], original_model['model.diffusion_model.output_blocks.11.1.transformer_blocks.0.attn1.to_v.weight']), 0)

In [62]:
model.load_state_dict(converted['diffusion'])

<All keys matched successfully>